In [ ]:
library(stringr)
library(data.table)
library(dplyr)
library(DescTools)

In [ ]:
source("eval_functions_refactored.R")

In [ ]:
covars <- fread("AoU_98K_covariates.tsv")
icd <- fread("icd_cohort_processed.txt")
colnames(icd)[2:1854] <- paste0("Phe_", str_replace(colnames(icd)[2:1854], "\\.", "_"))
head(colnames(icd))
sire <- fread("self_report_demographics.txt")

subpopPCs <- fread("2024-05-31_global_subpop_v2_gnomad-AoU-PCs-in-AOU.tsv")

In [ ]:
dir <- "comparisons_binary_refactored_naomit_10subpopPCs/" #using!
dir.create(dir)

In [ ]:
phecodes <- c("Phe_428_1", "Phe_427_21", "Phe_411_2", "Phe_411_3", "Phe_250_2")
methods <- c("LDpred", "PRScs", "PRScsx")

In [ ]:
colnames(dat)

In [ ]:
phecode <- "Phe_428_1"
method <- "LDpred"

score_data <- fread(paste0("computed_scores/", phecode, "_", 
        method, ".txt"))
score_num <- length(colnames(score_data)) - 4
dat <- icd %>% 
        select("person_id", all_of(phecode)) %>% 
        left_join(covars) %>% 
        left_join(score_data, by = c(person_id = "IID")) %>% 
        left_join(subpopPCs %>% 
                  select(!c(ancestry_pred_other, 
                            contains("prob"), 
                            contains("prev_global"))), 
                  by = c("person_id" = "s"))

pop <- "amr"
covs <- c("age", "is_male", paste0(toupper(pop), "_PC", 1:10))
#covs <- c("age", "is_male", paste0("PC", 1:10))
dat_anc <- dat[dat$ancestry_pred_other == pop, ]
dat_anc1 <- dat_anc %>% select(phecode, contains("SCORE"), all_of(covs)) %>% na.omit()

In [ ]:
i <- 10
score_col <- paste0("SCORE", i, "_SUM")
model <- glm(as.formula(paste(phecode, paste(c(paste0("scale(", 
                  score_col, ")"), covs), collapse = " + "), 
                  sep = " ~ ")), family = "binomial", data = dat_anc)
summary(model)
R2liab <- R2Liability(dat_anc, phecode, 10, covs)
R2liab

In [ ]:
R2liab <- R2Liability(dat_anc, phecode, NA, covs)
R2liab

In [ ]:
compare_method_popprev_subpopPCs <- function (phecode, dir, method, age_sex = FALSE, sire = NULL) 
{
    print(phecode)
    score_data <- fread(paste0("computed_scores/", phecode, "_", 
        method, ".txt"))
    score_num <- length(colnames(score_data)) - 4
    dat <- icd %>% 
        select("person_id", all_of(phecode)) %>% 
        left_join(covars) %>% 
        left_join(score_data, by = c(person_id = "IID")) %>% 
        left_join(subpopPCs %>% 
                  select(!c(ancestry_pred_other, 
                            contains("prob"), 
                            contains("prev_global"))), 
                  by = c("person_id" = "s"))
    
    if (!is.null(sire)) {
        dat <- dat %>% left_join(sire) %>% filter(unrel == 1)
    }
    else {
        dat <- dat %>% filter(unrel == 1)
    }
    grouping_var <- "ancestry_pred_other"
    grouping_name <- "ancestry"
    if (age_sex) {
        adj_level <- "PCagesex-adj"
    }else{
        adj_level <- "PC-adj"
    }
    
    cstat_file <- paste0(dir, phecode, "_", method, "_", grouping_name, 
        "_cstat_", adj_level, "_unrel.txt")
    beta_file <- paste0(dir, phecode, "_", method, "_", grouping_name, 
        "_beta_", adj_level, "_unrel.txt")
    R2liab_file <- paste0(dir, phecode, "_", method, "_", grouping_name, 
        "_R2liab_", adj_level, "_unrel.txt")
    print(cstat_file)
    
    for (pop in c("amr", "eur", "afr")) {
        if (age_sex) {
            covs <- c("age", "is_male", paste0(toupper(pop), "_PC", 1:20))
        }
        else {
            covs <- paste0(toupper(pop), "_PC", 1:10)   
        }
        dat_anc <- dat[dat[[grouping_var]] == pop, ]
        
        dat_anc <- dat_anc %>% select(phecode, contains("SCORE"), all_of(covs)) %>% na.omit()
        
        
        ncases <- sum(dat_anc[[phecode]] == T, na.rm = T)
        ncontrols <- sum(dat_anc[[phecode]] == F, na.rm = T)
        print(pop)
        print(ncases)
        if (ncases > 100) {
            
            baseline <- glm(as.formula(paste(phecode, paste0(covs, 
                collapse = "+"), sep = " ~ ")), family = "binomial", 
                data = dat_anc)
            baseline_R2liab <- R2Liability(dat_anc, phecode, 
                NA, covs)
            R2obs_values <- numeric(score_num)
            R2liab_values <- numeric(score_num)
            Cstat_values <- numeric(score_num)
            for (i in 1:score_num) {
                score_col <- paste0("SCORE", i, "_SUM")
                model <- glm(as.formula(paste(phecode, paste(c(paste0("scale(", 
                  score_col, ")"), covs), collapse = " + "), 
                  sep = " ~ ")), family = "binomial", data = dat_anc)
                write_beta_line(model, i, phecode, pop, beta_file)
                R2liab <- R2Liability(dat_anc, phecode, i, covs)
                R2obs_values[i] <- R2liab[1]
                R2liab_values[i] <- R2liab[2]
                Cstat_values[i] <- Cstat(model)
            }
            write(paste(phecode, pop, ncases, ncontrols, baseline_R2liab[1], 
                paste(R2obs_values, collapse = ";"), sep = ";"), 
                file = R2liab_file, append = TRUE)
            write(paste(phecode, pop, ncases, ncontrols, baseline_R2liab[2], 
                paste(R2liab_values, collapse = ";"), sep = ";"), 
                file = R2liab_file, append = TRUE)
            write(paste(phecode, pop, ncases, ncontrols, Cstat(baseline), 
                paste(Cstat_values, collapse = ";"), sep = ";"), 
                file = cstat_file, append = TRUE)
        }
    }
}

In [ ]:
for(method in methods){
    sapply(phecodes, compare_method_popprev_subpopPCs, dir=dir, method = method, age_sex = TRUE)
}